#### Importar bibliotecas e funcionalidades

In [1]:
import os
import pandas as pd

#### Visualisar colunas das planilhas

In [4]:
data_dir = r"C:\Users\diogo\OneDrive\Documents\Dev\Creating\Projetos\Python\Python Dashboard Equal\excel_data"

cols_by_archive = {}
for archive in os.listdir(data_dir):
    if archive[:5] == "clean":
        df = pd.read_excel(f"{data_dir}\\{archive}")
        cols_by_archive[archive] = df.columns.to_list()
        
print(cols_by_archive)

{'clean_dim_familia_produtos.xlsx': ['codigo_familia', 'descricaofamilia'], 'clean_dim_produtos.xlsx': ['codigo_produto', 'descricaoproduto', 'codigo_familia', 'custo_produto_unitario'], 'clean_dim_vendedor.xlsx': ['codigo_vendedor', 'nome_vendedor'], 'clean_fato_vendas.xlsx': ['codigo_contrato', 'data_venda', 'codigo_produto', 'filial_venda', 'quantidade', 'valor_monetario_total', 'valor_desconto', 'codigo_vendedor', 'codigo_cliente']}


#### Carregar planilhas

In [6]:
data_dir = r"C:\Users\diogo\OneDrive\Documents\Dev\Creating\Projetos\Python\Python Dashboard Equal\excel_data"
df_fato_vendas = pd.read_excel(f"{data_dir}\\clean_fato_vendas.xlsx")
df_dim_vendedor = pd.read_excel(f"{data_dir}\\clean_dim_vendedor.xlsx")
df_dim_produtos = pd.read_excel(f"{data_dir}\\clean_dim_produtos.xlsx")
df_dim_familia_produtos = pd.read_excel(f"{data_dir}\\clean_dim_familia_produtos.xlsx")

In [17]:
df_sums = df_fato_vendas.groupby("codigo_produto")[["valor_monetario_total", "quantidade"]].sum().reset_index()
df_sums = pd.merge(df_sums, df_dim_produtos, on='codigo_produto', how='inner')
df_sums = pd.merge(df_sums, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on='codigo_familia', how='inner')
display(df_sums)

,codigo_produto,valor_monetario_total,quantidade,descricaoproduto,codigo_familia,custo_produto_unitario,descricaofamilia
0,400,50904.47,30570,High-Quality Bedding Magenta #374,74,0.70,Bedding
1,245800,5492.45,115,Delicious Bedding White #541,74,24.73,Bedding
2,491000,19624.39,351,High-Quality Bedding Maroon #496,74,25.96,Bedding
3,560200,106980.66,101804,Fun Bedding Gold #556,74,0.50,Bedding
4,589600,12397.59,166,Modern Bedding Silver #733,74,36.08,Bedding
...,...,...,...,...,...,...,...
6970,3104504,124.00,1,Sustainable Hand Tools Green #12,CAMF,55.88,Hand Tools
6971,3104603,246.20,2,Beautiful Hand Tools Olive #972,CAMF,52.37,Hand Tools
6972,3104604,363.18,3,Sustainable Hand Tools Brown #881,CAMF,52.37,Hand Tools
6973,3104606,115.00,1,Delicious Hand Tools Teal #564,CAMF,52.37,Hand Tools


In [18]:
df_sums["lucro_produto"] = df_sums["valor_monetario_total"] - (df_sums["quantidade"] * df_sums["custo_produto_unitario"])
display(df_sums)

,codigo_produto,valor_monetario_total,quantidade,descricaoproduto,codigo_familia,custo_produto_unitario,descricaofamilia,lucro_produto
0,400,50904.47,30570,High-Quality Bedding Magenta #374,74,0.70,Bedding,29505.47
1,245800,5492.45,115,Delicious Bedding White #541,74,24.73,Bedding,2648.50
2,491000,19624.39,351,High-Quality Bedding Maroon #496,74,25.96,Bedding,10512.43
3,560200,106980.66,101804,Fun Bedding Gold #556,74,0.50,Bedding,56078.66
4,589600,12397.59,166,Modern Bedding Silver #733,74,36.08,Bedding,6408.31
...,...,...,...,...,...,...,...,...
6970,3104504,124.00,1,Sustainable Hand Tools Green #12,CAMF,55.88,Hand Tools,68.12
6971,3104603,246.20,2,Beautiful Hand Tools Olive #972,CAMF,52.37,Hand Tools,141.46
6972,3104604,363.18,3,Sustainable Hand Tools Brown #881,CAMF,52.37,Hand Tools,206.07
6973,3104606,115.00,1,Delicious Hand Tools Teal #564,CAMF,52.37,Hand Tools,62.63


#### Salvar nova planilha com dados tratados

In [10]:
df_sums.to_excel(f"{data_dir}\\df_fam_prod_venda.xlsx", index=False)

In [21]:
# Agrupar por familia
df_sums_cod_fam = df_sums.groupby("codigo_familia")["lucro_produto"].sum().reset_index()
display(df_sums_cod_fam)

,codigo_familia,lucro_produto
0,0,265849.97
1,03,239603.90
2,0333,443321.46
3,036,164297.98
4,1,430747.41
...,...,...
70,OJP,343687.40
71,OPP,270023.25
72,OTMO,150865.07
73,ROHO,202488.64


In [67]:
# Unir os dataframes
df_sums_cod_fam = pd.merge(df_sums_cod_fam, df_dim_familia_produtos[["codigo_familia", "descricaofamilia"]], on="codigo_familia", how='inner')
display(df_sums_cod_fam)

,codigo_familia,lucro_produto,descricaofamilia
0,0,265849.97,Ebooks
1,03,239603.90,Laundry Appliances
2,0333,443321.46,Pet Supplies
3,036,164297.98,Software
4,1,430747.41,Household Goods
...,...,...,...
70,OJP,343687.40,Hardware
71,OPP,270023.25,Necklaces
72,OTMO,150865.07,Software
73,ROHO,202488.64,Plants and Flowers


In [68]:
# Ordenar o dataframe pelo lucro_produto em ordem decrescente
df_sums_cod_fam = df_sums_cod_fam.sort_values(by='lucro_produto', ascending=True)
df_sums_cod_fam.reset_index(drop=True, inplace=True)
display(df_sums_cod_fam)

,codigo_familia,lucro_produto,descricaofamilia
0,CSF,3630995.22,Watches
1,CS,2305278.67,Bathroom Fixtures
2,DMA,1971676.55,Movies
3,27,1597602.27,Electronics
4,DMR,1246138.74,Automotive Repair
...,...,...,...
70,DSF,161059.29,Pet Food
71,OTMO,150865.07,Software
72,38,144503.30,Beverages
73,CRM,113752.11,Vehicle Accessories


In [40]:
# Salvar novo dataframe
# df_sums_cod_fam.to_excel(f"{data_dir}\\df_fam_top.xlsx", index=False)

In [41]:
# B. Para cada uma dessas 5 famílias, quais são os 3 produtos que tiveram maior
# contribuição para o lucro total acumulado?

In [69]:
df_sums_cod_fam_top_prod = pd.DataFrame()
x = int(input("Digite o valor: "))
df_sums_cod_fam_top_prod["codigo_familia"] = df_sums_cod_fam["codigo_familia"].head(x)
display(df_sums_cod_fam_top_prod)


,codigo_familia
0,CSF
1,CS
2,DMA
3,27
4,DMR


In [70]:
df_sums_cod_fam_top_prod = pd.merge(df_sums_cod_fam_top_prod, df_sums[["lucro_produto", "codigo_produto", "descricaoproduto", "codigo_familia"]], on="codigo_familia", how='inner')
display(df_sums_cod_fam_top_prod.head(5))

,codigo_familia,lucro_produto,codigo_produto,descricaoproduto
0,CSF,-8.39,1583700,Versatile Watches White #729
1,CSF,384.60,1704300,Comfortable Watches Yellow #231
2,CSF,13.61,1707500,Innovative Watches Teal #961
3,CSF,97.80,1721000,Delicious Watches White #947
4,CSF,146.70,1721300,Sustainable Watches Navy #376


In [71]:
df_sums_cod_fam_top_prod.sort_values(by='lucro_produto', ascending=False)
df_sums_cod_fam_top_prod.reset_index(drop=True, inplace=True)
display(df_sums_cod_fam_top_prod)

,codigo_familia,lucro_produto,codigo_produto,descricaoproduto
0,CSF,-8.39,1583700,Versatile Watches White #729
1,CSF,384.60,1704300,Comfortable Watches Yellow #231
2,CSF,13.61,1707500,Innovative Watches Teal #961
3,CSF,97.80,1721000,Delicious Watches White #947
4,CSF,146.70,1721300,Sustainable Watches Navy #376
...,...,...,...,...
3417,DMR,480.15,2837200,Durable Automotive Repair Magenta #750
3418,DMR,1074.03,2837300,Eco-Friendly Automotive Repair Purple #298
3419,DMR,24.15,2837400,Modern Automotive Repair Green #285
3420,DMR,27.66,2837500,Stylish Automotive Repair Pink #103


In [75]:
df_sums_cod_fam_top_prod_3c = pd.merge(df_sums_cod_fam_top_prod, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on="codigo_familia", how='inner')
display(df_sums_cod_fam_top_prod_3c)

,codigo_familia,lucro_produto,codigo_produto,descricaoproduto,descricaofamilia
0,CSF,-8.39,1583700,Versatile Watches White #729,Watches
1,CSF,384.60,1704300,Comfortable Watches Yellow #231,Watches
2,CSF,13.61,1707500,Innovative Watches Teal #961,Watches
3,CSF,97.80,1721000,Delicious Watches White #947,Watches
4,CSF,146.70,1721300,Sustainable Watches Navy #376,Watches
...,...,...,...,...,...
3417,DMR,480.15,2837200,Durable Automotive Repair Magenta #750,Automotive Repair
3418,DMR,1074.03,2837300,Eco-Friendly Automotive Repair Purple #298,Automotive Repair
3419,DMR,24.15,2837400,Modern Automotive Repair Green #285,Automotive Repair
3420,DMR,27.66,2837500,Stylish Automotive Repair Pink #103,Automotive Repair


In [76]:
y = int(input("Digite o valor: "))
df_sums_cod_fam_top_prod_3c = df_sums_cod_fam_top_prod_3c.groupby('codigo_familia').apply(lambda x: x.nlargest(y, 'lucro_produto')).reset_index(drop=True)
display(df_sums_cod_fam_top_prod_3c)

,codigo_familia,lucro_produto,codigo_produto,descricaoproduto,descricaofamilia
0,27,84630.47,783800,Sustainable Electronics Blue #485,Electronics
1,27,66529.11,985300,Practical Electronics Navy #139,Electronics
2,27,54260.46,783700,Luxurious Electronics Blue #551,Electronics
3,CS,18626.65,2922403,High-Quality Bathroom Fixtures Maroon #813,Bathroom Fixtures
4,CS,14193.61,2644610,Versatile Bathroom Fixtures Magenta #907,Bathroom Fixtures
5,CS,13558.91,2922402,Powerful Bathroom Fixtures Silver #163,Bathroom Fixtures
6,CSF,25375.13,2644209,Durable Watches Gray #812,Watches
7,CSF,19397.89,2644208,Delicious Watches Navy #308,Watches
8,CSF,19321.04,2508326,Beautiful Watches Purple #770,Watches
9,DMA,578967.66,2730100,Fun Movies Red #7,Movies


In [ ]:
df_sums_cod_fam_top_prod_3c.to_excel(f"{data_dir}\\df_fam_top_by_products.xlsx", index=False)

In [45]:
# C. Ao avaliar esses produtos, é possível perceber se os que geram maior lucro
# são também os mais vendidos?
# D. Dica: Crie colunas que comparem custo unitário, preço de venda e descontos
# aplicados para calcular a margem de lucro.

In [46]:
# import os
# import pandas as pd

# data_dir = r"C:\Users\diogo\OneDrive\Documents\Dev\Creating\Projetos\Python\Python Dashboard Equal\excel_data"

# # Lista de arquivos para ler
# files = [
#     "clean_fato_vendas.xlsx",
#     "clean_dim_vendedor.xlsx",
#     "clean_dim_produtos.xlsx",
#     "clean_dim_familia_produtos.xlsx"
# ]

# # Lendo os arquivos em um loop e armazenando em um dicionário
# dfs = {}
# for file in files:
#     name = os.path.splitext(file)[0]
#     dfs[name] = pd.read_excel(os.path.join(data_dir, file))

# # Realizando o agrupamento e cálculos
# df_sums = dfs["clean_fato_vendas"].groupby("codigo_produto")[["valor_monetario_total", "quantidade"]].sum().reset_index()
# df_sums = pd.merge(df_sums, dfs["clean_dim_produtos"], on='codigo_produto', how='inner')
# df_sums["lucro_produto"] = df_sums["valor_monetario_total"] - (df_sums["quantidade"] * df_sums["custo_produto_unitario"])

# df_sums_cod_fam = df_sums.groupby("codigo_familia")["lucro_produto"].sum()
# df_sums_cod_fam = pd.merge(df_sums_cod_fam, dfs["clean_dim_familia_produtos"][["codigo_familia", "descricaofamilia"]], on="codigo_familia", how='inner')
# df_sums_cod_fam = df_sums_cod_fam.sort_values(by='lucro_produto', ascending=False).reset_index(drop=True)

# # Exibindo os resultados
# display(df_sums)
# display(df_sums_cod_fam)


In [ ]:
import pandas as pd

# Obter o valor de entrada
x = int(input("Digite o valor para limitar o número de linhas do DataFrame: "))

# Criar DataFrame com as primeiras x linhas de df_sums_cod_fam
df_sums_cod_fam_top_prod = df_sums_cod_fam.head(x)[["codigo_familia"]]

# Juntar com df_sums para obter as informações adicionais
df_sums_cod_fam_top_prod = pd.merge(df_sums_cod_fam_top_prod, df_sums[["lucro_produto", "codigo_produto", "descricaoproduto", "codigo_familia"]], on="codigo_familia", how='inner')

# Classificar o DataFrame por lucro_produto
df_sums_cod_fam_top_prod.sort_values(by='lucro_produto', ascending=False, inplace=True)

# Resetar o índice
df_sums_cod_fam_top_prod.reset_index(drop=True, inplace=True)

# Juntar com df_dim_familia_produtos para obter a descrição da família
df_sums_cod_fam_top_prod_3c = pd.merge(df_sums_cod_fam_top_prod, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on="codigo_familia", how='inner')

# Obter o valor de entrada para limitar o número de linhas por grupo
y = int(input("Digite o valor para limitar o número de linhas por grupo: "))

# Selecionar os maiores valores de lucro_produto por grupo
df_sums_cod_fam_top_prod_3c = df_sums_cod_fam_top_prod_3c.groupby('codigo_familia').apply(lambda x: x.nlargest(y, 'lucro_produto')).reset_index(drop=True)

# Exibir o DataFrame resultante
display(df_sums_cod_fam_top_prod_3c)